In [7]:
test_input = """
???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
"""

with open('day12.txt', 'rt') as f: raw_input = f.read()

In [165]:
from itertools import combinations
from queue import deque

OPERATIONAL = '.'
DAMAGED = '#'
UNKNOWN = '?'

def damaged_group(conditions: iter):
    group_sizes = []
    count = 0
    for cond in conditions:
        if cond == DAMAGED:
            count += 1
        else:
            if count:
                group_sizes.append(count)
                count = 0
    if count:
        group_sizes.append(count)
    return group_sizes

def find_arrangements_bf(record: str) -> list:
    conditions, group_sizes = record.strip().split()
    conditions = conditions
    group_sizes = list(map(int, group_sizes.split(',')))
    
    total_damaged = sum(group_sizes)
    remain_damaged = total_damaged - conditions.count(DAMAGED)
    damaged_able_positions = [i for i, cond in enumerate(conditions) if cond == UNKNOWN]

    assumption_conditions = [cond if cond == DAMAGED else OPERATIONAL for cond in conditions]
    arrangements = set()
    for positions in combinations(damaged_able_positions, remain_damaged):
        clarified_conditions = assumption_conditions[::]
        for position in positions:
            clarified_conditions[position] = DAMAGED
        
        if damaged_group(clarified_conditions) == group_sizes:
            arrangements.add(''.join(clarified_conditions))
    return arrangements

def solve1(raw_input: str):
    return sum(len(find_arrangements(row)) for row in raw_input.strip().splitlines())

def find_arrangements_2(record: str, factor: int = 1) -> list:
    conditions, group_sizes = record.strip().split()
    conditions = '?'.join([conditions for _ in range(factor)])
    group_sizes = list(map(int, group_sizes.split(','))) * factor
    
    arrangements = []
    stacks = deque()
    stacks.append((0, conditions, group_sizes))
    steps = 0
    while stacks:
        steps += 1
        index, conds, gsizes  = stacks.popleft()
        if not gsizes:
            if DAMAGED not in conds[index:]:
                arrangements.append(conds.replace(UNKNOWN, OPERATIONAL))
            continue
        count = 0
        expect_size = gsizes[0]
        split_case = None
        for cindex, ccond in enumerate(conds[index:], index):
            if not count and ccond == OPERATIONAL:
                continue
            elif ccond == DAMAGED:
                count += 1
            elif ccond == OPERATIONAL:
                break
            else:
                if count == expect_size:
                    # Unknown have to be operational
                    conds = conds[:cindex] + OPERATIONAL + conds[cindex + 1:]
                    break
                else:
                    # If not start, split into another case
                    if count == 0:
                        nconds = conds[:cindex] + OPERATIONAL + conds[cindex + 1:]
                        split_case = (cindex + 1, nconds, gsizes)
                    #  Unknown have to be damaged
                    conds = conds[:cindex] + DAMAGED + conds[cindex + 1:]
                    count += 1
#         print(conds)
        if split_case:
            stacks.append(split_case)
        if count == expect_size:
            stacks.append((cindex + 1, conds, gsizes[1:]))
        else:
            continue

    return arrangements

def find_arrangements(conditions, group_sizes) -> list:
    arrangements = 0
    stacks = deque()
    stacks.append((conditions, group_sizes))
    steps = 0
    while stacks:
        steps += 1
        conds, gsizes  = stacks.popleft()
        if not gsizes:
            if DAMAGED not in conds:
                arrangements += 1
            continue
        count = 0
        expect_size = gsizes[0]
        split_case = None
        for cindex, ccond in enumerate(conds):
            if not count and ccond == OPERATIONAL:
                continue
            elif ccond == DAMAGED:
                count += 1
            elif ccond == OPERATIONAL:
                break
            else:
                if count < expect_size:
                    # If not start, split into another case
                    if count == 0:
                        split_case = (conds[cindex + 1:], gsizes)
                    #  Unknown have to be damaged
                    count += 1
                else:
                    break
#         print(count, conds, gsizes)
        if split_case:
            stacks.append(split_case)
        if count == expect_size:
            stacks.append((conds[cindex + 1:], gsizes[1:]))
        else:
            continue

    return arrangements

def find_arrangements_factor(record: str, factor: int = 1) -> list:
    conditions, group_sizes = record.strip().split()
    conditions = '?'.join([conditions for _ in range(factor)])
    group_sizes = list(map(int, group_sizes.split(','))) * factor
    return find_arrangements(conditions, group_sizes)

def solve2(raw_input: str):
    return sum(len(find_arrangements_2(row, factor=5)) for row in raw_input.strip().splitlines())


In [161]:
len(find_arrangements_bf('??????#??????????#??? 7,1,7,1'))

12

In [170]:
find_arrangements_number(raw_input.strip().splitlines()[2], 5)

32

In [ ]:
for index, row in enumerate(raw_input.strip().splitlines()):
    print(index, len(find_arrangements_2(row, factor=5)))

0 1428
1 87846
2 32
3 712044
